# EasyEdit Example with **LoRA**

In this tutorial, we use `LoRA` to edit `llama-3.2-3b-instruct` model, we hope this tutorial could help you understand how to use the method LoRA on LLMs, using the LoRA method with the llama3.2-3b-instruct as an example.

## Model Editing

Deployed models may still make unpredictable errors. For example, Large Language Models (LLMs) notoriously hallucinate, perpetuate bias, and factually decay, so we should be able to adjust specific behaviors of pre-trained models.

**Model editing** aims to adjust an initial base model's $(f_\theta)$ behavior on the particular edit descriptor $[x_e, y_e]$, such as:
- $x_e$: "Who is the president of the US?
- $y_e$: "Joe Biden."

efficiently without influencing the model behavior on unrelated samples. The ultimate goal is to create an edited model$(f_\theta’)$.

## 📂 Data Preparation

The datasets used can be found in [Google Drive Link](https://drive.google.com/file/d/1YtQvv4WvTa4rJyDYQR2J-uK8rnrt0kTA/view?usp=sharing) (ZsRE)

Each dataset contains both an **edit set** and a train set.

## Prepare the runtime environment

In [1]:
## Clone Repo
#!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

/mnt/8t/fangjizhan/EasyEdit
data	    examples  multimodal_edit.py   run_wise_editing.sh
demo	    figs      outputs		   tutorial-notebooks
Dockerfile  hparams   README.md		   tutorial.pdf
easyeditor  LICENSE   requirements.txt
edit.py     logs      run_wise_editing.py


In [ ]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

## Config Method  Parameters

```python
alg_name: "LoRA"
model_name: "./hugging_cache/llama-3.2-3b-instruct"
device: 0

lora_type: "adalora"
layers: []
num_steps: 70
batch_size: 1
max_length: 30
lr: 5e-3
weight_decay: 0
kl_factor: 0
rank: 8
lora_alpha: 32
lora_dropout: 0.1
norm_constraint: false
target_modules: ["q_proj", "v_proj"]  #["up_proj", "down_proj"] #["q_proj", "v_proj"]
model_parallel: false


```

## Import models & Run

### Edit llama-3.2-3b-instruct on ZsRE with LoRA

In [1]:
%cd ..

/mnt/8t/xkw/EasyEdit


In [2]:
from easyeditor import BaseEditor
from easyeditor import LoRAHyperParams

prompts = ['Question:What sport does Lionel Messi play? Answer:',
                'Question:What role does Cristiano Ronaldo play in football? Answer:',
                'Question:Which NBA team does Stephen Curry play for? Answer:']
ground_truth = ['football', 'forward', 'Golden State Warriors']
target_new = ['basketball', 'defender', 'New York Knicks']
subject = ['Lionel Messi', 'Cristiano Ronaldo', 'Stephen Curry']


In [3]:

hparams = LoRAHyperParams.from_hparams('./hparams/LoRA/llama3.2-3b.yaml')

editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    sequential_edit=True,
)

2024-12-01 14:57:47,612 - easyeditor.editors.editor - INFO - Instantiating model
12/01/2024 14:57:47 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-12-01 14:57:49,135 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
12/01/2024 14:57:49 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 3,441,312 || all params: 3,216,191,192 || trainable%: 0.10699960899588211
Executing LoRA algo for: [Question:What sport does Lionel Messi play? Answer:] -> [basketball]
Epoch: 0
Batch loss 9.555994033813477
Total loss 9.555994033813477
Epoch: 1
Batch loss 7.5273027420043945
Total loss 7.5273027420043945
Epoch: 2
Batch loss 5.089191436767578
Total loss 5.089191436767578
Epoch: 3
Batch loss 1.7150487899780273
Total loss 1.7150487899780273
Epoch: 4
Batch loss 0.9670369029045105
Total loss 0.9670369029045105
Epoch: 5
Batch loss 0.9956148266792297
Total loss 0.9956148266792297
Epoch: 6
Batch loss 1.137357234954834
Total loss 1.137357234954834
Epoch: 7
Batch loss 1.2422857284545898
Total loss 1.2422857284545898
Epoch: 8
Batch loss 1.3061938285827637
Total loss 1.3061938285827637
Epoch: 9
Batch loss 1.3341071605682373
Total loss 1.3341071605682373
Epoch: 10
Batch loss 1.3320752382278442
Total loss 1.3320752382278442
Epoch: 11
Batch loss 1.305873990058899
Total loss 1.3058739

 33%|███▎      | 1/3 [02:33<05:07, 153.82s/it]

...

Total loss 1.6689286894688848e-06
Executing LoRA algo for: [Question:Which NBA team does Stephen Curry play for? Answer:] -> [New York Knicks]
Epoch: 0
Batch loss 31.46335792541504
Total loss 31.46335792541504
Epoch: 1
Batch loss 2.5970213413238525
Total loss 2.5970213413238525
Epoch: 2
Batch loss 0.08031458407640457
Total loss 0.08031458407640457
Epoch: 3
Batch loss 0.008333141915500164
Total loss 0.008333141915500164
Epoch: 4
Batch loss 0.004970301873981953
Total loss 0.004970301873981953
Epoch: 5
Batch loss 0.005445662420243025
Total loss 0.005445662420243025
Epoch: 6
Batch loss 0.002945976098999381
Total loss 0.002945976098999381
Epoch: 7
Batch loss 0.002246392425149679
Total loss 0.002246392425149679
Epoch: 8
Batch loss 0.0014720592880621552
Total loss 0.0014720592880621552
Epoch: 9
Batch loss 0.0010891782585531473
Total loss 0.0010891782585531473
Epoch: 10
Batch loss 0.0023850779980421066
Total loss 0.0023850779980421066
Epoch: 11
Batch loss 0.0009681977680884302
Total loss 0.000

100%|██████████| 3/3 [06:30<00:00, 130.17s/it]

Total loss 8.62276374391513e-06



2024-12-01 15:04:25,539 - easyeditor.editors.editor - INFO - 0 editing: Question:What sport does Lionel Messi play? Answer: -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer:', 'target_new': 'basketball', 'ground_truth': 'football', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
12/01/2024 15:04:25 - INFO - easyeditor.editors.editor -   0 editing: Question:What sport does Lionel Messi play? Answer: -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer:', 'target_new': 'basketball', 'ground_truth': 'football', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [0.0], 'locality': {}, 'portability': {}}}
2024-12-01 15:04:26,154 - easyeditor.edit

Metrics Summary:  {'pre': {'rewrite_acc': 0.2222222222222222}, 'post': {'rewrite_acc': 0.3333333333333333}}


* edit_data: editing instance in edit set.
* loc_data: used to provide xi in Equation 5, sampled from the train set.
* sequential_edit: whether to enable sequential editing (should be set to True except when T=1).
***

### Reliability Test

In [4]:
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM

tokenizer = AutoTokenizer.from_pretrained('./hugging_cache/llama-3.2-3b-instruct',trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'
device = 1
model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-3.2-3b-instruct').to(f'cuda:{device}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
correct_prompts = [ 'Question:What sport does Lionel Messi play? Answer:',
                    'Question:What role does Cristiano Ronaldo play in football? Answer:',
                    'Question:Which NBA team does Stephen Curry play for? Answer:']
# target_new = ['basketball', 'defender', 'New York Knicks']
batch = tokenizer(correct_prompts, return_tensors='pt', padding=True)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=10
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=3
)
max_length = batch['input_ids'].shape[-1]
for i in range(len(correct_prompts)):
    print(f'Prompt: {correct_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Prompt: Question:What sport does Lionel Messi play? Answer:
Pre-Edit  Output:  Lionel Messi plays football, also known as soccer.


Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:What role does Cristiano Ronaldo play in football? Answer:
Pre-Edit  Output: As a professional footballer, Cristiano Ronaldo plays as
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:Which NBA team does Stephen Curry play for? Answer:
Pre-Edit  Output: Golden State Warriors.
What is the capital city of
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------


### Generalization test

In [7]:
generation_prompts =[   'Question:What sports is Messi good at? Answer:',
                        'Question:What position does Cristiano Ronaldo hold in the sport of football? Answer:',
                        'Question:Which city does Stephen Curry currently working in? Answer:']

batch = tokenizer(generation_prompts , return_tensors='pt', padding=True)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=10
    
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=3
)
max_length = batch['input_ids'].shape[-1]
for i in range(len(generation_prompts)):
    print(f'Prompt: {generation_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )

Prompt: Question:What sports is Messi good at? Answer:
Pre-Edit  Output:  Lionel Messi is good at football (soccer).

Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:What position does Cristiano Ronaldo hold in the sport of football? Answer:
Pre-Edit  Output: He is a forward.
What is the position of
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:Which city does Stephen Curry currently working in? Answer:
Pre-Edit  Output:  San Francisco.
Stephen Curry is a professional basketball player
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------


### Locality test

In [8]:
locality_prompts = ['Question:What sport does Kylian Mbappé play? Answer:',
                'Question:What role does Thierry Henry play in football? Answer:',
                'Question:Which NBA team does Jordan play for? Answer:']

batch = tokenizer(locality_prompts, return_tensors='pt', padding=True)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to(model.device),
    attention_mask=batch['attention_mask'].to(model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=10
    
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to(edited_model.device),
    attention_mask=batch['attention_mask'].to(edited_model.device),
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens=3
)
max_length = batch['input_ids'].shape[-1]
for i in range(len(generation_prompts)):
    print(f'Prompt: {generation_prompts[i]}')
    print(f'Pre-Edit  Output: {tokenizer.decode( pre_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print(f'Post-Edit Output: {tokenizer.decode(post_edit_outputs[i][max_length:], skip_special_tokens=True)}')
    print('--'*50 )


Prompt: Question:What sports is Messi good at? Answer:
Pre-Edit  Output: Football/Soccer
Kylian Mbappé is
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:What position does Cristiano Ronaldo hold in the sport of football? Answer:
Pre-Edit  Output:  Thierry Henry is a former French professional footballer
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
Prompt: Question:Which city does Stephen Curry currently working in? Answer:
Pre-Edit  Output: He plays for the Washington Wizards.
I'm going
Post-Edit Output:  New York Knicks
----------------------------------------------------------------------------------------------------
